In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Project import Project

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from Scripts.neuralnets.NNTrainingUtils import train_autoencoder
from Scripts.neuralnets.NNPreinstalledModelSelection import *
from Scripts.neuralnets.NNTrainingUtils import train, kfold_train
from Scripts.data.SSVEPDataset import SSVEPDataset
from Scripts.data.SSVEPDataloader import SSVEPDataloader

In [4]:
ngpu = 1; device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

***

In [ ]:
def read_data(dataframe):
    return torch.tensor(pd.read_csv(dataframe['path'], sep = ' ', header = None, dtype = float).values)

def dataVisualization(X1, X2, save = False, filename = None):
    fig, axs = plt.subplots(nrows = 2, ncols = 1)
    
    fig.set_size_inches(12, 5)
    fig.subplots_adjust(hspace = 0)
    
    for ax in axs:
        ax.xaxis.set_major_formatter(plt.NullFormatter())
        ax.yaxis.set_major_formatter(plt.NullFormatter())
    
    axs[0].imshow(X1, cmap = 'gray'); axs[0].set_title("Original")
    axs[1].imshow(X2, cmap = 'gray'); axs[1].set_title("Autoencoder's Output")
    
    if save: fig.savefig(filename)
    
    return fig, axs

def EncoderVisualization(model, data, save = False, filename = None):
    fig, ax = plt.subplots(nrows = 1, ncols = 1)
    fig.set_size_inches(15, 15)
    fig.tight_layout()
    ax.xaxis.set_major_formatter(plt.NullFormatter())
    ax.yaxis.set_major_formatter(plt.NullFormatter())
    
    image = model.encode(data).detach()[0]
    image = (image.transpose(0, 2).transpose(0, 1).numpy())
    ax.imshow(image)
    if save: fig.savefig(filename)
        
    return fig, ax

***

In [5]:
preprocessing = 'cca'
signal_length = '512'
data = pd.read_csv((Project.output_dir / ('SSVEPDataset_'+signal_length+'.csv')).__str__(), dtype = 'str')
dataset = SSVEPDataset(Project, preprocessing, data, signal_length)

In [6]:
batch_size = 64
dataloaders, dataset_sizes = SSVEPDataloader(dataset, batch_size)
print(dataset_sizes)

{'train': 2184, 'val': 546}


In [7]:
from Models.Autoencoder1 import Autoencoder

In [8]:
model = Autoencoder(kernel_n = 5, stride = 1).to(device)
summary(model, (1, 16, int(signal_length)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 3, 16, 512]              36
              ReLU-2           [-1, 3, 16, 512]               0
         MaxPool2d-3  [[-1, 3, 16, 256], [-1, 3, 16, 256]]               0
       BatchNorm2d-4           [-1, 3, 16, 256]               6
       MaxUnpool2d-5           [-1, 3, 16, 512]               0
   ConvTranspose2d-6           [-1, 1, 16, 512]              34
Total params: 76
Trainable params: 76
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.03
Forward/backward pass size (MB): 1151.28
Params size (MB): 0.00
Estimated Total Size (MB): 1151.31
----------------------------------------------------------------


In [9]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)

In [10]:
model, stats = train_autoencoder(Project, model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs = 100)

[24/Sep/2020 13:13:48] INFO - Epoch    Stage       Loss  

[24/Sep/2020 13:13:49] INFO - 1/100    Training    0.63  
[24/Sep/2020 13:13:49] INFO -          Validation  0.30  
[24/Sep/2020 13:13:49] INFO - 2/100    Training    0.24  
[24/Sep/2020 13:13:49] INFO -          Validation  0.20  
[24/Sep/2020 13:13:50] INFO - 3/100    Training    0.18  
[24/Sep/2020 13:13:50] INFO -          Validation  0.16  
[24/Sep/2020 13:13:51] INFO - 4/100    Training    0.15  
[24/Sep/2020 13:13:51] INFO -          Validation  0.13  
[24/Sep/2020 13:13:51] INFO - 5/100    Training    0.13  
[24/Sep/2020 13:13:51] INFO -          Validation  0.12  
[24/Sep/2020 13:13:52] INFO - 6/100    Training    0.11  
[24/Sep/2020 13:13:52] INFO -          Validation  0.11  
[24/Sep/2020 13:13:52] INFO - 7/100    Training    0.10  
[24/Sep/2020 13:13:52] INFO -          Validation  0.10  
[24/Sep/2020 13:13:53] INFO - 8/100    Training    0.09  
[24/Sep/2020 13:13:53] INFO -          Validation  0.09  
[24/Sep/2020 

***

Visualization

In [ ]:
exemplo = next(iter(dataloaders['train']))[0][0].unsqueeze(0).to(device)

In [ ]:
output = model(exemplo).detach()

In [ ]:
fig, ax = dataVisualization(exemplo.squeeze()[:, :100].cpu(), output.squeeze()[:, :100].cpu(), save = False, filename = "16x64_comparison.png")

In [ ]:
fig, ax = EncoderVisualization(model.cpu(), exemplo.cpu(), save = False, filename = '16x16_encoded.png')

***

Classification

In [ ]:
class HybridModel(nn.Module):
    def __init__(self, classifier, encoder):
        super(HybridModel, self).__init__()
        self.encoder = encoder
        self.classifier = classifier
        
    def forward(self, x):
        with torch.set_grad_enabled(False):
            x = self.encoder(x)
        
        x = self.classifier(x)
        
        return x

In [ ]:
model_type = 'resnet'; model_size = '50'; num_classes = 5; model_name = model_type + model_size
classifier = model_selection(model_name, num_classes)
encoder = model.encoder
hybrid = HybridModel(classifier, encoder).to(device)

In [ ]:
optimizer = optim.SGD(hybrid.parameters(), lr = 0.001, momentum = 0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)
criterion = nn.CrossEntropyLoss()

In [ ]:
hybrid, stats = kfold_train(Project, hybrid, dataset, criterion, optimizer, num_epochs = 20, num_folds = 5)

***